In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.manifold import TSNE

%matplotlib inline



In [12]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
avisos.drop("nombre_area", axis = 1, inplace=True)
avisos.head()

,idaviso,titulo,denominacion_empresa,titulo_uppercase,pide_hombre,pide_mujer,ingles,experiencia,paquete_office,pide_excel,...,tipo_de_trabajo_Por Horas,tipo_de_trabajo_Primer empleo,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,VENTOR,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Wurth Argentina S.A,True,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
2,1000872556,Operarios de Planta - Rubro Electrodomésticos,ELECTRO OUTLET SRL,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Wurth Argentina S.A,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Wurth Argentina S.A,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True


In [13]:
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")


postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

vistas.head()

,idaviso,idpostulante,cantidad,total_vistas
0,18,BolNL,2,28
1,1738407,BolNL,1,28
2,1795995,BolNL,2,28
3,1804592,BolNL,1,28
4,1804723,BolNL,1,28


In [14]:
#seteo el peso como cantidad de vistas del postulante al anuncio/total de vistas del postulante
vistas["cantidad"] = vistas["cantidad"]/vistas["total_vistas"]
vistas = vistas.drop("total_vistas",axis=1)
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,1738407,BolNL,0.035714
2,1795995,BolNL,0.071429
3,1804592,BolNL,0.035714
4,1804723,BolNL,0.035714


In [15]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
no_postulaciones=pd.merge(no_postulaciones,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
no_postulaciones=pd.merge(no_postulaciones,avisos,on='idaviso')

no_postulaciones.head()

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Por Horas,tipo_de_trabajo_Primer empleo,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,1112430536,EexaOz,0,38.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1112430536,owaWw0Z,0,20.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,1112430536,PmGPRd1,0,19.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,1112430536,bO4q324,0,25.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,1112430536,MVd5dlr,0,30.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [25]:
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

labels = postulaciones["sepostulo"]
data = postulaciones[features]

data.head()

,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,Master_Abandonado,Master_En Curso,Master_Graduado,...,tipo_de_trabajo_Por Horas,tipo_de_trabajo_Primer empleo,tipo_de_trabajo_Teletrabajo,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,47.0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,34.0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,39.0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,39.0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,32.0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [ ]:

model = TSNE(n_components=3, random_state=0)

tsne_data = model.fit_transform(data)
tsne_ = np.vstack(tsne_data.T, labels).T
df = pd.DataFrame(tsne_)


sns.FacetGrid(df,hue = "label").map(plt.scatter, features)